In [15]:
import pandas as pd
from pathlib import Path
from geopy.distance import geodesic
import numpy as np

In [ ]:
#Connect to the database
conn_string = "host='localhost' dbname='glacier_data' user='postgres' password='REDACTED'"
#password has been removed for security reasons
conn = psycopg2.connect(conn_string)
print("Database opened successfully")
cursor = conn.cursor();
query = 'select * from yourdatabase'
cursor.execute(query)
colnames = [desc[0] for desc in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns=colnames)
cursor.close()
print('Database closed successfully!')

In [16]:
glaciers_raw = pd.read_csv(Path("../Data/AK_1985_to_2000_overall_glacier_covered_area.csv"))
glaciers_raw.head()

,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Name,GN_area,GN_comp_ye
0,RGI60-01.00002,G213332E63404N,20090703,-9999999,-146.668,63.404,1,2,0.558,1713,...,1197,0,0,0,0,9,9,NaN,0.5922,2000
1,RGI60-01.00003,G213920E63376N,20090703,-9999999,-146.080,63.376,1,2,1.685,1609,...,2106,0,0,0,0,9,9,NaN,1.8045,2000
2,RGI60-01.00004,G213880E63381N,20090703,-9999999,-146.120,63.381,1,2,3.681,1273,...,4175,0,0,0,0,9,9,NaN,3.6765,2000
3,RGI60-01.00005,G212943E63551N,20090703,-9999999,-147.057,63.551,1,2,2.573,1494,...,2981,0,0,0,0,9,9,NaN,2.6064,2000
4,RGI60-01.00006,G213756E63571N,20090703,-9999999,-146.244,63.571,1,2,10.470,1201,...,10518,0,0,0,0,9,9,NaN,7.7220,2000


In [17]:
#glaciers only
glaciers=glaciers_raw[glaciers_raw['Form'] == 0]
#clarifying column names and converting to Imperial 
glaciers['Id'] = glaciers_raw['GLIMSId']
glaciers['Height (m)'] = glaciers_raw['Zmax'] - glaciers_raw['Zmin']
glaciers['Height (ft)'] = glaciers['Height (m)'] * 3.281
glaciers['Length (ft)'] = glaciers_raw['Lmax'] * 3.281
glaciers['Area (mi^2)'] = glaciers_raw['GN_area'] / 2.59
glaciers['Year'] = glaciers_raw['GN_comp_ye']
glaciers['Lat'] = glaciers_raw['CenLat']
glaciers['Lon'] = glaciers_raw['CenLon']
#removing excess columns and sorting by Id
glaciers=glaciers.sort_values(by=["Id", "Year"]).reset_index()
glaciers=glaciers[['Year', 'Lat', 'Lon', 'Area (mi^2)', 'Height (ft)', 'Length (ft)']]
glaciers.head()

,Year,Lat,Lon,Area (mi^2),Height (ft),Length (ft)
0,1986,53.881,-167.0,0.031969,2723.23,6949.158
1,1988,53.881,-167.0,0.031969,2723.23,6949.158
2,1990,53.881,-167.0,0.031969,2723.23,6949.158
3,1992,53.881,-167.0,0.031969,2723.23,6949.158
4,1994,53.881,-167.0,0.031969,2723.23,6949.158


In [18]:
#weather data for Juneau, Fairbanks, Valdez, Anchorage
weather1 = pd.read_csv(Path("../Data/Weather_data_1940_to_2024.csv"))
weather1.head()

,Date,Monthly Total Precipitation (in),Monthly Average Mean Temperature (degF),Monthly Total Snowfall (in),City
0,1940-01,NaN,NaN,NaN,Juneau
1,1940-02,NaN,NaN,NaN,Juneau
2,1940-03,NaN,NaN,NaN,Juneau
3,1940-04,NaN,NaN,NaN,Juneau
4,1940-05,NaN,NaN,NaN,Juneau


In [19]:
#weather data for Delta Junction, Kodiak, Ketchikan
weather2 = pd.read_csv(Path("../Data/three more cities.csv"))
weather2.head()

,Date,Monthly Total Precipitation (in),Monthly Average Mean Temperature (degF),Monthly Total Snowfall (in),City,ID
0,1940-01,NaN,NaN,NaN,Delta Junction,USW00026415
1,1940-02,NaN,NaN,NaN,Delta Junction,USW00026415
2,1940-03,NaN,NaN,NaN,Delta Junction,USW00026415
3,1940-04,NaN,NaN,NaN,Delta Junction,USW00026415
4,1940-05,NaN,NaN,NaN,Delta Junction,USW00026415


In [20]:
#merging two data sets
weather=pd.merge(weather1, weather2, on=['Date','Monthly Total Precipitation (in)','Monthly Average Mean Temperature (degF)','Monthly Total Snowfall (in)','City'], how='outer')
weather.head()

,Date,Monthly Total Precipitation (in),Monthly Average Mean Temperature (degF),Monthly Total Snowfall (in),City,ID
0,1940-01,1.34,-1.60,27.1,Fairbanks,NaN
1,1940-01,5.91,21.47,33.0,Valdez,NaN
2,1940-01,6.52,40.50,NaN,Ketchikan,USW00025325
3,1940-01,12.75,36.84,9.2,Kodiak,USW00025501
4,1940-01,NaN,NaN,NaN,Anchorage,NaN


In [21]:
#coordinates of these cities
weather.loc[weather['City'] == 'Juneau', ['Lat', 'Lon']] = [58.3005, -134.433304]
weather.loc[weather['City'] == 'Fairbanks', ['Lat', 'Lon']] = [64.8401, -147.7200]
weather.loc[weather['City'] == 'Anchorage', ['Lat', 'Lon']] = [61.2176, -149.8997]
weather.loc[weather['City'] == 'Valdez', ['Lat', 'Lon']] = [61.1309, -146.3499]
weather.loc[weather['City'] == 'Kodiak', ['Lat', 'Lon']] = [57.7905031, -152.4451733]
weather.loc[weather['City'] == 'Ketchikan', ['Lat', 'Lon']] = [55.3453476, -131.6817362]
weather.loc[weather['City'] == 'Delta Junction', ['Lat', 'Lon']] = [64.0619727, -145.7870219]
#grouping by year
weather['Year'] = weather['Date'].astype(str).str[:4]
weather = weather[weather['Year'].between('1986', '2020')]
weather=weather.sort_values(by=["City", "Year"]).reset_index()
weather=weather[['Lat','Lon', 'Year', 'Monthly Average Mean Temperature (degF)', 'Monthly Total Precipitation (in)', 'Monthly Total Snowfall (in)']]
annual_weather = weather.groupby('Year').mean()
annual_weather.reset_index(inplace=True)
annual_weather.head()

,Year,Lat,Lon,Monthly Average Mean Temperature (degF),Monthly Total Precipitation (in),Monthly Total Snowfall (in)
0,1986,60.383846,-144.045262,36.834868,4.392237,9.062500
1,1987,60.383846,-144.045262,38.816627,4.920617,7.982667
2,1988,60.383846,-144.045262,37.976790,4.935432,8.583333
3,1989,60.383846,-144.045262,37.051786,4.166429,9.774684
4,1990,60.383846,-144.045262,36.198452,4.153494,13.654667


In [22]:
#writing to csv for future use
annual_weather.to_csv("../Data/weathercombined.csv", index=False, header=True)

In [23]:
# Function to calculate the closest city using geodesic
def find_closest_city(Lat_glacier, Lon_glacier, cities_df, distance_threshold=3):
    closest_distance = float('inf')  # Start with an infinitely large distance
    closest_city = None
    for _, row in cities_df.iterrows():
        # Calculate the difference in Lat/Lon
        Lat_diff = abs(Lat_glacier - row['Lat'])
        Lon_diff = abs(Lon_glacier - row['Lon'])
        # Check if the Lat/Lon differences are within the 3-degree threshold
        if Lat_diff <= distance_threshold and Lon_diff <= distance_threshold:
            # Compute the geodesic distance between the glacier and the city
            distance = geodesic((Lat_glacier, Lon_glacier), (row['Lat'], row['Lon'])).km
            # If the distance is the smallest so far, update the closest city
            if distance < closest_distance:
                closest_distance = distance
                closest_city = row
    return closest_city

In [24]:
#note: this code will take about 10 mins to run
closest_lats = []
closest_lons = []
# For each glacier, find the closest city
def find_closest_city(Lat_glacier, Lon_glacier, cities_df, distance_threshold=3):
    # Calculate the differences in latitudes and longitudes
    lat_diff = abs(cities_df['Lat'] - Lat_glacier)
    lon_diff = abs(cities_df['Lon'] - Lon_glacier)
    # Filter cities that are within the 3-degree threshold in both lat and lon
    cities_within_threshold = cities_df[(lat_diff <= distance_threshold) & (lon_diff <= distance_threshold)]
    if cities_within_threshold.empty:
        return np.nan, np.nan
    # Compute the geodesic distance for each filtered city
    cities_within_threshold['distance'] = cities_within_threshold.apply(
        lambda row: geodesic((Lat_glacier, Lon_glacier), (row['Lat'], row['Lon'])).km, axis=1)
    # Find the city with the minimum distance
    closest_city = cities_within_threshold.loc[cities_within_threshold['distance'].idxmin()]
    return closest_city['Lat'], closest_city['Lon']
# Apply the optimized function to the 'glaciers' DataFrame to find the closest city for each glacier
glaciers['closest_lat'], glaciers['closest_lon'] = zip(*glaciers.apply(
    lambda row: find_closest_city(row['Lat'], row['Lon'], annual_weather), axis=1))
glaciers.head(10)

,Year,Lat,Lon,Area (mi^2),Height (ft),Length (ft),closest_lat,closest_lon
0,1986,53.881,-167.0,0.031969,2723.23,6949.158,NaN,NaN
1,1988,53.881,-167.0,0.031969,2723.23,6949.158,NaN,NaN
2,1990,53.881,-167.0,0.031969,2723.23,6949.158,NaN,NaN
3,1992,53.881,-167.0,0.031969,2723.23,6949.158,NaN,NaN
4,1994,53.881,-167.0,0.031969,2723.23,6949.158,NaN,NaN
5,1996,53.881,-167.0,0.031969,2723.23,6949.158,NaN,NaN
6,1998,53.881,-167.0,0.107375,2723.23,6949.158,NaN,NaN
7,2000,53.881,-167.0,0.168880,2723.23,6949.158,NaN,NaN
8,2002,53.881,-167.0,0.124402,2723.23,6949.158,NaN,NaN
9,2004,53.881,-167.0,0.143861,2723.23,6949.158,NaN,NaN


In [27]:
#cleaning the new data
close_glaciers=glaciers.dropna(subset=['closest_lat'])
close_glaciers=close_glaciers.dropna(subset=['closest_lon'])
close_glaciers['GLat'] = close_glaciers['Lat']
close_glaciers['GLon'] = close_glaciers['Lon']
close_glaciers['Lat'] =close_glaciers['closest_lat']
close_glaciers['Lon'] =close_glaciers['closest_lon']
close_glaciers=close_glaciers[['Year', 'Lat', 'Lon', 'Area (mi^2)', 'Height (ft)', 'Length (ft)']].reset_index(drop=True)
close_glaciers.head(10)

,Year,Lat,Lon,Area (mi^2),Height (ft),Length (ft)
0,1986,60.383846,-144.045262,0.120579,1086.011,1594.566
1,1988,60.383846,-144.045262,0.118494,1086.011,1594.566
2,1990,60.383846,-144.045262,0.115367,1086.011,1594.566
3,1992,60.383846,-144.045262,0.109807,1086.011,1594.566
4,1994,60.383846,-144.045262,0.103205,1086.011,1594.566
5,1996,60.383846,-144.045262,0.082703,1086.011,1594.566
6,1998,60.383846,-144.045262,0.072625,1086.011,1594.566
7,2000,60.383846,-144.045262,0.078880,1086.011,1594.566
8,2002,60.383846,-144.045262,0.079923,1086.011,1594.566
9,2004,60.383846,-144.045262,0.084093,1086.011,1594.566


In [28]:
#shape of this dataset
close_glaciers.shape

(70724, 6)

In [29]:
#lost about 3 fourths of the data, but thats still 70k rows of data
annual_weather['Year'] = annual_weather['Year'].fillna(0).astype('int64')
merged=pd.merge(annual_weather, close_glaciers, on=['Lat','Lon','Year'], how='outer')
merged=merged[['Year', 'Monthly Average Mean Temperature (degF)', 'Monthly Total Precipitation (in)', 'Monthly Total Snowfall (in)', 'Area (mi^2)', 'Height (ft)', 'Length (ft)']]
merged.head()

,Year,Monthly Average Mean Temperature (degF),Monthly Total Precipitation (in),Monthly Total Snowfall (in),Area (mi^2),Height (ft),Length (ft)
0,1986,36.834868,4.392237,9.0625,0.120579,1086.011,1594.566
1,1986,36.834868,4.392237,9.0625,0.012510,488.869,-29.529
2,1986,36.834868,4.392237,9.0625,0.005212,331.381,-29.529
3,1986,36.834868,4.392237,9.0625,0.039961,193.579,-29.529
4,1986,36.834868,4.392237,9.0625,0.167490,1099.135,1991.567


In [30]:
#exporting for UML use
merged.to_csv("../Data/UMLprep.csv", index=False, header=True)

In [ ]:
#..